# Take-Home Assignment: Linear Aeroelastic Galloping

---

Welcome to the final assignment for **Introduction to Aeroelastic Instabilities with Jupyter**! 🎉

In this assignment, you'll explore **galloping**, an interesting aeroelastic instability that affects slender structures like power lines, bridge cables, and stay cables when exposed to wind. Unlike flutter, which requires coupling between multiple degrees of freedom, galloping is a **single-degree-of-freedom instability** driven entirely by the aerodynamic characteristics of the cross-section.

### What is Galloping?

Galloping is a type of aeroelastic instability characterized by large-amplitude, low-frequency oscillations perpendicular to the wind direction. You may have seen videos of power lines swaying dramatically in the wind, or cables on bridges oscillating with surprisingly large amplitudes. This is galloping!

The key physical mechanism is that **certain cross-sectional shapes** (rectangles, D-sections, ice-covered cables) generate aerodynamic forces that can **destabilize** the structure when it moves. The secret lies in how the aerodynamic force changes with angle of attack.

### Learning Objectives

By completing this assignment, you will be able to:

1. **Derive** the equation of motion for the linear galloping oscillator using Lagrange's equations
2. **Understand** the physical mechanism behind galloping instability and the role of aerodynamic force slope
3. **Calculate** the critical galloping speed and explain the stability criterion
4. **Analyze** system stability using root locus diagrams
5. **Create** V-g and V-f diagrams to visualize how stability changes with airspeed
6. **Simulate** the time-domain response at different conditions
7. **Investigate** how system parameters affect galloping behavior

*Let's get started!*

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Enable interactive plots - change "widget" to "inline" if running in Google Colab
%matplotlib widget

## 1. Physical Setup and Aerodynamic Model

---

Consider the 2D cross-section of a bluff body exposed to a uniform flow, as shown below. You can imagine this as a slice of a long rectangular prism. The body is free to oscillate in the vertical direction (heave) as it is suspended by a linear spring, it is subjected to aerodynamic forces from the wind, and its motion is resited by a linear dashpot representing the damping of the structure.

<figure align="center">
<img src="../figures/galloping_oscillator.svg" style="width:50%">
<figcaption align = "center"> Galloping oscillator with heave degree of freedom. </figcaption>
</figure>

The system parameters are:

- Height $H$ (dimension perpendicular to the flow)
- Mass $m$ (kg)
- Spring stiffness $k$ (N/m)
- Dashpot damping coefficient $c$ (Ns/m)

As mentioned, the body can only move in the **vertical direction** (heave, $h$), perpendicular to the incoming flow. The flow has:
- Velocity $V$ (m/s)
- Density $\rho$ (kg/m³)

### Aerodynamic Force Model

You should be familiar with be familiar with the concept of the **effective angle of attack**, with the **heave approximation**, and with the assumption of **quasi-steady aerodynamics**. We introduced these concepts when we discussed the heave-only typical section in notebook 2, so feel free to review that notebook if you need a refresher!

Similarly to the heave-only typical section, also our bluff body is subjected to a time-varying aerodynamic force $f_h(t)$ along the vertical direction that depends on the angle of attack $\alpha(t)$. Given the configuration of the system, the angle of attack corresponds to the effective angle of attack $\alpha_h(t)$, which in turn depends on the heave velocity $\dot{h}(t)$:

$$
f_h(t) = \frac{1}{2}\rho V^2 H c_{f_h}(\alpha(t))\\[1ex]
\alpha(t) = \alpha_h(t) \approx -\frac{\dot{h}(t)}{V},
$$

where $c_{f_h}(\alpha)$ is the aerodynamic force coefficient in the vertical direction. Analogously to the case of the airfoil, this force coefficient depends on the angle of attack $\alpha$ and it relates the aerodynamic force to the dynamic pressure and the reference length, which in this case is the height $H$ of the body.

Unlike streamlined airfoils where the flow remains attached and generates lift through circulation, bluff bodies like rectangular cylinders, D-sections, or ice-accreted cables experience significant **flow separation** even at small angles of attack. As the body moves or the angle of attack changes, the separation points shift along the body's surface, creating complex vortex shedding patterns and pressure distributions that can produce forces in unexpected directions.

Crucially, for certain cross-sectional shapes and angle of attack ranges, the transverse force coefficient $c_{f_h}(\alpha)$ exhibits a counterintuitive behavior: as the angle of attack increases, the force can actually *decrease*, resulting in regions where $\frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha} < 0$. This negative slope is the aerodynamic fingerprint that makes galloping possible. In reality, the relationship between force and angle of attack is **highly nonlinear**. Experimental data show that $c_{f_h}(\alpha)$ typically requires polynomial fits with multiple terms (e.g., $c_{f_h}(\alpha) = A_1\alpha + A_3\alpha^3 + A_5\alpha^5 + ...$) to capture the complete behavior.

However, for the linear galloping analysis, we focus solely on the behavior near the equilibrium position by retaining only the **first-order term** of this expansion: $c_{f_h}(\alpha) \approx \frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0} \alpha$. This linearization is valid for small oscillations and allows us to derive an analytical expression for the critical galloping speed. The higher-order nonlinear terms, though neglected here, ultimately govern the amplitude of the limit cycle oscillations that occur in the post-critical regime. This topic lies beyond the scope of our linear analysis but, in case you are interested, it is treated comprehensively in [this book](https://onlinelibrary.wiley.com/doi/book/10.1002/9781118756478).

By retaining only the linear term, we obtain the following expression for the aerodynamic force:

$$f_h(t) \approx \frac{1}{2}\rho V^2 H \frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0} \alpha_h(t).$$

If the expression $\frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0}$ looks daunting, don't worry! For this assignment, it'll just correspond to a constant coefficient representing the slope of the aerodynamic force coefficient at the equilibrium position. This slope is the key parameter that determines whether galloping can occur or not.

## 2. Derivation of the Equation of Motion using Lagrange's Equations

---

Your first task is to derive the equation of motion for the linear galloping oscillator using Lagrange's equations. This will allow us to understand how the system's parameters and the aerodynamic force slope influence stability. As a reference for the derivation, you can look at notebook 3, where we derived the equations of motion for the heave-pitch typical section. Since the galloping oscillator has only one degree of freedom, the derivation will be much simpler, but the steps are the same.

Starting from the Lagrange's equations:

$$\frac{\mathrm{d}}{\mathrm{d}t}\left(\frac{\partial \mathscr{L}}{\partial \dot{\boldsymbol{q}}}\right) - \frac{\partial \mathscr{L}}{\partial \boldsymbol{q}} = \boldsymbol{Q}$$

where $\boldsymbol{q}$ is the vector of generalized coordinates, $\mathscr{L}$ is the Lagrangian of the system, and $\boldsymbol{Q}$ represents the vector of non-conservative generalized forces, you can follow these steps to derive the equation of motion:

1. Define the generalized coordinates (only one in this case!)
2. Express the kinetic energy $T$ of the system
3. Express the potential energy $U$ of the system
4. Form the Lagrangian $\mathscr{L} = T - U$
5. Apply Lagrange's equations to derive the left hand side of the equation of motion
6. Determine the generalized forces (only one actually) through the principle of virtual work:
$$\boldsymbol{Q} = \frac{\partial\left(\delta W\right)}{\partial\left(\delta \boldsymbol{q}\right)}$$
7. Assemble the equation of motion and show that it can be expressed as an homogeneous second-order ODE.

Complete this markdown cell with the derivation, and make sure to explain each step clearly.

## 3. Critical Galloping Speed and Stability Analysis

---

In the previous section, you should find that the equation of motion for the linear galloping oscillator can be expressed as:

$$m\ddot{h} + \left(c + \frac{1}{2}\rho V H \frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0}\right)\dot{h} + kh = 0.$$

Now your task is to analyze the damping term $\left(c + \frac{1}{2}\rho V H \frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0}\right)$ and determine the critical galloping speed $V_g$ at which the system transitions from stable to unstable behavior. Remember that a positive damping coefficient indicates that energy is being dissipated (thus giving a stabilizing effect), while a negative damping coefficient indicates that energy is being added to the system (thus giving a destabilizing effect).

Express the condition for neutral stability (i.e., zero damping) and derive an expression for the critical galloping speed $V_g$ in terms of the system parameters and the aerodynamic force slope. Assuming that all system parameters ($m$, $c$, $k$) are positive, can you explain what is the condition on $\frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0}$ for galloping to occur?

Complete this markdown cell with your derivation and explanation.

## 4. System Setup and Critical Galloping Speed Calculation

---

Let's define our baseline system parameters.

In [2]:
m = 50.0            # mass (kg)
k = 2000.0          # spring stiffness (N/m)
c = 100.0           # structural damping coefficient (Ns/m)
H = 1.              # height of the body (m)
rho = 1.225         # air density (kg/m³)
dcfh_dalpha = -2.7  # aerodynamic force slope (dimensionless, negative!)

Now, using the expression you derived for the critical galloping speed $V_g$, calculate the critical speed for this system. Complete the code cell below and print the result.

In [3]:
## TODO: Calculate the critical galloping speed V_g using the formula derived in the previous section and print the result
# ...

## 5. Root Locus Analysis

---

Analogously to what we did in notebook 2 and 3, now your task is to analyze the stability of the system using the root locus diagram. In particular, you should plot the root locus as a function of the airspeed $V$ and identify the critical speed at which the poles cross into the right half-plane, indicating the onset of instability.

First of all, you need to convert the equation of motion into a state-space representation. Then you'll need to define a function to compute the system matrix for a given airspeed $V$, define a range of airspeeds to analyze, calculate the eigenvalues of the system matrix for each airspeed, and finally plot the root locus diagram. Feel free to reuse the code from notebook 2 and 3, but make sure to adapt it to the single-degree-of-freedom case of the galloping oscillator.

Complete this markdown cell with the derivation of the state-space representation. Successively, complete the code cell below to perform the root locus analysis and plot the results.

In [4]:
# TODO: define function to compute system matrix for a given speed V
# ...

# TODO: define range of speeds to analyze
# ...

# TODO: initialize arrays to store real and imaginary parts of eigenvalues, based on the number of eigenvalues that you expect
# ...

# TODO: loop over speeds, compute system matrix, compute eigenvalues, and store real and imaginary parts in the arrays
# ...

# TODO: create a scatter plot of the real and imaginary parts of the eigenvalues and color code the points based on the speed V
# ...

Now use this markdown cell to analyze the root locus diagram you obtained. What do you observe? How do the real and imaginary parts of the eigenvalues change with increasing airspeed? Can you explain why they change in that way?

## 6. V-g and V-f Diagrams

---

Now your task is to create V-g and V-f diagrams for the system based on the results of your root locus analysis. Remember that the V-g diagram plots the damping ratio $\zeta$ as a function of the airspeed $V$, while the V-f diagram plots the frequency $\omega$ of oscillation as a function of $V$. The formulas to compute these quantities from the eigenvalues $\lambda$ are:

* Damping Ratio: $\zeta = \frac{-\mathbb{Re}(\lambda)}{\sqrt{\left(\mathbb{Re}(\lambda)\right)^2 + \left(\mathbb{Im}(\lambda)\right)^2}}$
* Frequency: $\omega = \mathbb{Im}(\lambda)$

Complete the code cell below to compute $\zeta$ and $\omega$ for each airspeed and plot the V-g and V-f diagrams.


In [5]:
# TODO: initialize arrays to store frequency and damping ratio
# ...

# TODO: iterate over the previously computed real and imaginary parts to calculate frequency and damping ratio
# ...

# TODO: create two subplots - one for V-g plot (damping ratio vs velocity) and one for V-f plot (frequency vs velocity)
# ...

Discuss the trends you observe in the V-g and V-f diagrams. How do damping ratio and frequency change with increasing airspeed? And how do you explain these trends?

## 7. Time-Domain Simulations

---

Now your task is to simulate the actual time response of the system at three different conditions:
1. **Well-damped** ($V = 0.5 V_g$): Below critical speed, system must be stable
2. **Subcritical** ($V = 0.95 V_g$): Close to critical speed, system must be marginally stable (almost undamped oscillations)
3. **Supercritical** ($V = 1.1 V_g$): Above critical speed, system must be unstable (galloping!)

Analogously to what we did in notebook 2 and 3, you can use the `scipy.integrate.solve_ivp` function to perform the time-domain simulations. Make sure to set appropriate initial conditions and simulate for a sufficient amount of time to observe the system's behavior. Complete the code cell below to perform the simulations and plot the time response for each case.

In [6]:
# TODO: define the function that returns dx/dt = A * x
# ...

# TODO: define flight speeds to test
# ...

# TODO: define simulation parameters
# ...

# TODO: iterate over speeds and plot time response for each case
# ...

Use this markdown cell to briefly analyze your results. Do you observe the expected behavior in each case?

## 8. Effect of System Parameters

---

Now your task is to investigate how changing different system parameters affects the galloping behavior. You are going to study the effect of the following parameters:
1. **Mass** ($m$)
2. **Stiffness** ($k$)
3. **Structural damping** ($c$)
4. **Aerodynamic force slope** $\left(\frac{\mathrm{d}c_{f_h}}{\mathrm{d}\alpha}\bigg|_{\alpha_0}\right)$

For each parameter, you'll create V-g and V-f diagrams with three different values, in order to observe how the critical galloping speed and the stability characteristics change. You are free to try and choose three different values for each parameter that according to your judgment, so that the changes in the diagrams are clearly visible.

Use a different code cell for each parameter that you investigate, and create a markdown cell below each code cell to discuss the results.

In [7]:
# --- Mass variation ---
# TODO: define mass values to test
# ...

# TODO: Create figure with two subplots
# ...

# TODO: Iterate over mass values
# ...

# TODO: initialize arrays to store frequency and damping ratio
# ...

# TODO: loop over speeds, compute system matrix, compute eigenvalues, and compute frequency and damping ratio from real and imaginary parts of the eigenvalues
# Careful: this will be a nested loop!
# ...

# TODO: plot V-g plot V-f plot in the outer loop (mass value loop)
# ...

Discuss mass results here.

In [8]:
# --- Stiffness variation ---
# TODO: define stiffness values to test
# ...

# TODO: Create figure with two subplots
# ...

# TODO: Iterate over stiffness values
# ...

# TODO: initialize arrays to store frequency and damping ratio
# ...

# TODO: loop over speeds, compute system matrix, compute eigenvalues, and compute frequency and damping ratio from real and imaginary parts of the eigenvalues
# Careful: this will be a nested loop!
# ...

# TODO: plot V-g plot V-f plot in the outer loop (stiffness value loop)
# ...

Discuss stiffness results here.

In [9]:
# --- Structural damping variation ---
# TODO: define damping values to test
# ...

# TODO: Create figure with two subplots
# ...

# TODO: Iterate over damping values
# ...

# TODO: initialize arrays to store frequency and damping ratio
# ...

# TODO: loop over speeds, compute system matrix, compute eigenvalues, and compute frequency and damping ratio from real and imaginary parts of the eigenvalues
# Careful: this will be a nested loop!
# ...

# TODO: plot V-g plot V-f plot in the outer loop (damping value loop)
# ...

Discuss structural damping results here.

In [10]:
# --- Aerodyamic force coefficient slope variation ---
# TODO: define slope values to test
# ...

# TODO: Create figure with two subplots
# ...

# TODO: Iterate over slope values
# ...

# TODO: initialize arrays to store frequency and damping ratio
# ...

# TODO: loop over speeds, compute system matrix, compute eigenvalues, and compute frequency and damping ratio from real and imaginary parts of the eigenvalues
# Careful: this will be a nested loop!
# ...

# TODO: plot V-g plot V-f plot in the outer loop (slope value loop)
# ...

Discuss force coefficient slope results here.

## 9. Enjoy and Happy Galloping! 🎉

---

Hope you have fun with this assignment and that it helps you consolidate what you have learned in the course. Once you have completed all the tasks, please send the notebook file via email to:
[francesco.mitrotta@uc3m.es](mailto:francesco.mitrotta@uc3m.es)

If you have any questions, please don't hesitate to reach out!
